# DS 7331 - Machine Learning - Course Project
Team: Allison Roderick, Jayson Barker, Sandesh Ojha, Brandon Croom

Lab Number: Mini Project

## Model Creation/Parameters - 50pts

In [1]:
# Import libraries for data analysis
from sklearn.model_selection import ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt

In [2]:
#Run the original EDA notebook. 
#NOTE: This will display all information from the original notebook
#Simply double-click the left side to close
%run ./JB_AR_BC_SO_Project1_Skinny.ipynb

Initial Shape:  (2974335, 49)
Deduplication Shape:  (2974335, 49)
No Duplicate Data Found
['MapQuest' 'MapQuest-Bing' 'Bing']
['Calm' 'SW' 'SSW' 'WSW' 'WNW' 'NW' 'West' 'NNW' 'NNE' 'South' 'North'
 'Variable' 'SE' 'SSE' 'ESE' 'East' 'NE' 'ENE' 'E' 'W' nan 'S' 'VAR'
 'CALM' 'N']
['Calm' 'SW' 'SSW' 'WSW' 'WNW' 'NW' 'W' 'NNW' 'NNE' 'S' 'N' 'VAR' 'SE'
 'SSE' 'ESE' 'E' 'NE' 'ENE' nan 'CALM']
(2974335, 41)
ID                        object
Severity                   int64
Start_Time                object
End_Time                  object
Start_Lat                float64
Start_Lng                float64
Distance(mi)             float64
Side                      object
City                      object
County                    object
State                     object
Zipcode                   object
Country                   object
Timezone                  object
Airport_Code              object
Temperature(F)           float64
Wind_Chill(F)            float64
Humidity(%)              float64
Pr

C:\Users\croomb\AppData\Local\Continuum\anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
#Define helper functions
def getMedianTemp(month):
    return medianTemp[month]

def getMedianPrecip(month):
    return medianPrecip [month]

def getMedianPressure(month):
    return medianPressure [month]

def getMedianWindChill(month):
    return medianWindChill [month]

def getMedianVis(month):
    return medianVis [month]

def getMedianWindSpeed(month):
    return medianWindSpeed [month]

def getMedianHumidity(month):
    return medianHumidity[month]

In [83]:
# Copy the dataframe from the original as a precaution
df2 = df.copy()

# Convert accident severity to a binary for logistic regression. Severity 1-2 accidents coded as 0, 
# Severity 3-4 accidents coded as 1
df2.Severity.replace([0, 1, 2, 3, 4], [0, 0, 0, 1, 1], inplace=True)

# View recoded variable
df2['Severity'].value_counts() 

0    1994378
1     979957
Name: Severity, dtype: int64

In [85]:
#Build the test/train split. Use a split of 80/20
#Get the values for Y
y = df2['Severity'].values
#Delete off the values for Y so they don't go into the model
del df2['Severity']
#get everything else that we want to predict off of
x = df2.values

X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=0)

MemoryError: 

In [37]:
#Convert the train/test splits back to dataframes so we can impute missing data
df_X_train = pd.DataFrame.from_records(X_train)
df_X_test = pd.DataFrame.from_records(X_test)
df_X_train.columns = df2.columns
df_X_test.columns = df2.columns

In [ ]:
print("Temp " + df_X_train['TemperatureF'].isnull().sum())
print("Precip " + df_X_train['Precipitationin'].isnull().sum())
print("Pressure " + df_X_train['Pressurein'].isnull().sum())
print("Wind Chill: " + df_X_train['Wind_ChillF'].isnull().sum())
print("Viz: " + df_X_train['Visibilitymi'].isnull().sum())
print("Wind Speed: " + df_X_train['Wind_Speedmph'].isnull().sum())
print("Humidity: " + df_X_train['Humidity'].isnull().sum())

In [67]:
medianTemp = df_X_train.groupby('Start_Month')['TemperatureF'].median(skipna=True)
medianPrecip = df_X_train.groupby('Start_Month')['Precipitationin'].median(skipna=True)
medianPressure = df_X_train.groupby('Start_Month')['Pressurein'].median(skipna=True)
medianWindChill = df_X_train.groupby('Start_Month')['Wind_ChillF'].median(skipna=True)
medianVis = df_X_train.groupby('Start_Month')['Visibilitymi'].median(skipna=True)
medianWindSpeed = df_X_train.groupby('Start_Month')['Wind_Speedmph'].median(skipna=True)
medianHumidity = df_X_train.groupby('Start_Month')['Humidity'].median(skipna=True)
#df_X_train['Precipitationin'] = df_X_train.groupby('Start_Month')['Precipitationin'].transform(lambda x: x.fillna(x.median()))
#df_X_train['Pressurein'] = df_X_train.groupby('Start_Month')['Pressurein'].transform(lambda x: x.fillna(x.median()))
#df_X_train['Wind_ChillF'] = df_X_train.groupby('Start_Month')['Wind_ChillF'].transform(lambda x: x.fillna(x.median()))
#df_X_train['Visibilitymi'] = df_X_train.groupby('Start_Month')['Visibilitymi'].transform(lambda x: x.fillna(x.median()))
#df_X_train['Wind_Speedmph'] = df_X_train.groupby('Start_Month')['Wind_Speedmph'].transform(lambda x: x.fillna(x.median()))
#df_X_train['Humidity'] = df_X_train.groupby('Start_Month')['Humidity'].transform(lambda x: x.fillna(x.median()))

In [82]:
df_X_train.loc[pd.isnull(df_X_train['TemperatureF']),'TemperatureF'] = df_X_train.Start_Month.apply(getMedianTemp)
df_X_train.loc[pd.isnull(df_X_train['Precipitationin']),'Precipitationin'] = df_X_train.Start_Month.apply(getMedianPrecip)
df_X_train.loc[pd.isnull(df_X_train['Pressurein']),'Pressurein'] = df_X_train.Start_Month.apply(getMedianPressure)
df_X_train.loc[pd.isnull(df_X_train['Wind_ChillF']),'Wind_ChillF'] = df_X_train.Start_Month.apply(getMedianWindChill)
df_X_train.loc[pd.isnull(df_X_train['Visibilitymi']),'Visibilitymi'] = df_X_train.Start_Month.apply(getMedianVis)
df_X_train.loc[pd.isnull(df_X_train['Visibilitymi']),'Wind_Speedmph'] = df_X_train.Start_Month.apply(getMedianWindSpeed)
df_X_train.loc[pd.isnull(df_X_train['Visibilitymi']),'Wind_Speedmph'] = df_X_train.Start_Month.apply(getMedianHumidity)


In [44]:
print("Temp " + df_X_train['TemperatureF'].isnull().sum())
print("Precip " + df_X_train['Precipitationin'].isnull().sum())
print("Pressure " + df_X_train['Pressurein'].isnull().sum())
print("Wind Chill: " + df_X_train['Wind_ChillF'].isnull().sum())
print("Viz: " + df_X_train['Visibilitymi'].isnull().sum())
print("Wind Speed: " + df_X_train['Wind_Speedmph'].isnull().sum())
print("Humidity: " + df_X_train['Humidity'].isnull().sum())

11768

In [14]:
# transform categories into dummy numerics
cols_to_transform = ['Side','Country','Weather_Condition','Sunrise_Sunset','State']
df2 = pd.get_dummies(df2, columns = cols_to_transform )

In [12]:
df2.describe()

,Severity,Start_Lat,Start_Lng,Distancemi,TemperatureF,Wind_ChillF,Humidity,Pressurein,Visibilitymi,Wind_Speedmph,Precipitationin,Start_Year,Start_Day,Start_Hour,End_Year,End_Day,End_Hour,Time_Durationmin,Side_,Side_L,Side_R,Country_US,Weather_Condition_Blowing Dust,Weather_Condition_Blowing Dust / Windy,Weather_Condition_Blowing Sand,Weather_Condition_Blowing Snow,Weather_Condition_Blowing Snow / Windy,Weather_Condition_Clear,Weather_Condition_Cloudy,Weather_Condition_Cloudy / Windy,Weather_Condition_Drizzle,Weather_Condition_Drizzle / Windy,Weather_Condition_Drizzle and Fog,Weather_Condition_Dust Whirls,Weather_Condition_Fair,Weather_Condition_Fair / Windy,Weather_Condition_Fog,Weather_Condition_Fog / Windy,Weather_Condition_Freezing Rain,Weather_Condition_Freezing Rain / Windy,Weather_Condition_Funnel Cloud,Weather_Condition_Hail,Weather_Condition_Haze,Weather_Condition_Haze / Windy,Weather_Condition_Heavy Blowing Snow,Weather_Condition_Heavy Drizzle,Weather_Condition_Heavy Freezing Drizzle,Weather_Condition_Heavy Freezing Rain,Weather_Condition_Heavy Ice Pellets,Weather_Condition_Heavy Rain,Weather_Condition_Heavy Rain / Windy,Weather_Condition_Heavy Rain Showers,Weather_Condition_Heavy Sleet,Weather_Condition_Heavy Smoke,Weather_Condition_Heavy Snow,Weather_Condition_Heavy Snow / Windy,Weather_Condition_Heavy Snow with Thunder,Weather_Condition_Heavy T-Storm,Weather_Condition_Heavy T-Storm / Windy,Weather_Condition_Heavy Thunderstorms and Rain,Weather_Condition_Heavy Thunderstorms and Snow,Weather_Condition_Heavy Thunderstorms with Small Hail,Weather_Condition_Ice Pellets,Weather_Condition_Light Blowing Snow,Weather_Condition_Light Drizzle,Weather_Condition_Light Drizzle / Windy,Weather_Condition_Light Fog,Weather_Condition_Light Freezing Drizzle,Weather_Condition_Light Freezing Fog,Weather_Condition_Light Freezing Rain,Weather_Condition_Light Freezing Rain / Windy,Weather_Condition_Light Hail,Weather_Condition_Light Haze,Weather_Condition_Light Ice Pellets,Weather_Condition_Light Rain,Weather_Condition_Light Rain / Windy,Weather_Condition_Light Rain Shower,Weather_Condition_Light Rain Shower / Windy,Weather_Condition_Light Rain Showers,Weather_Condition_Light Rain with Thunder,Weather_Condition_Light Sleet,Weather_Condition_Light Snow,Weather_Condition_Light Snow / Windy,Weather_Condition_Light Snow Grains,Weather_Condition_Light Snow Shower,Weather_Condition_Light Snow Showers,Weather_Condition_Light Snow and Sleet,Weather_Condition_Light Snow and Sleet / Windy,Weather_Condition_Light Snow with Thunder,Weather_Condition_Light Thunderstorm,Weather_Condition_Light Thunderstorms and Rain,Weather_Condition_Light Thunderstorms and Snow,Weather_Condition_Low Drifting Snow,Weather_Condition_Mist,Weather_Condition_Mostly Cloudy,Weather_Condition_Mostly Cloudy / Windy,Weather_Condition_N/A Precipitation,Weather_Condition_Overcast,Weather_Condition_Partial Fog,Weather_Condition_Partial Fog / Windy,Weather_Condition_Partly Cloudy,Weather_Condition_Partly Cloudy / Windy,Weather_Condition_Patches of Fog,Weather_Condition_Rain,Weather_Condition_Rain / Windy,Weather_Condition_Rain Shower,Weather_Condition_Rain Showers,Weather_Condition_Sand,Weather_Condition_Sand / Dust Whirlwinds,Weather_Condition_Sand / Dust Whirlwinds / Windy,Weather_Condition_Scattered Clouds,Weather_Condition_Shallow Fog,Weather_Condition_Showers in the Vicinity,Weather_Condition_Sleet,Weather_Condition_Small Hail,Weather_Condition_Smoke,Weather_Condition_Smoke / Windy,Weather_Condition_Snow,Weather_Condition_Snow / Windy,Weather_Condition_Snow Grains,Weather_Condition_Snow Showers,Weather_Condition_Snow and Sleet,Weather_Condition_Snow and Sleet / Windy,Weather_Condition_Snow and Thunder,Weather_Condition_Squalls,Weather_Condition_Squalls / Windy,Weather_Condition_T-Storm,Weather_Condition_T-Storm / Windy,Weather_Condition_Thunder,Weather_Condition_Thunder / Windy,Weather_Condition_Thunder / Wintry Mix / Windy,Weather_Condition_Thunder and Hail / Windy,Weather_Co

## Model Advantages - 10pts

## Interpretation - 30pts

## SVM Insight - 10pts